In [ ]:
from copy import deepcopy

class color:
    PURPLE   = '\033[95m'
    CYAN     = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE     = '\033[94m'
    GREEN    = '\033[92m'
    YELLOW   = '\033[93m'
    RED      = '\033[91m'
    WHITE    = '\u001b[37;1m'
    BOLD     = '\033[1m'
    
    UNDERLINE = '\033[4m'
    
    BrightCyan    = u'\u001b[36;1m'
    BgBrightBlack = u'\u001b[40;1m'
    BgBrightRed   = u'\u001b[41;1m'
    BgBrightGreen = u'\u001b[42;1m'
    BgBrightYellow = u'\u001b[43;1m'
    BgBrightBlue  = u'\u001b[44;1m'
    BgBrightMagenta = u'\u001b[45;1m'
    BgBrightCyan  = u'\u001b[46;1m'
    BgBrightWhite = u'\u001b[47;1m'                    
    
    END = '\033[0m'
    
def disk(disk):
    return color.BrightCyan+' '+color.WHITE+disk+' '+color.END
    
class TOH:
    def __init__(self):
        self.state=[[],[],[]]
        self.nodeNumber=0
        self.status='idle'
        self.neighbours=[]
        self.parent=None
        self.children=[]#For BFS
        self.point=10



def node_evaluation(node):
    largest=0
    l=[]
    for peg in initialState:
        if len(peg)>0:
            l.append(max(peg))

    largest=max(l)
    #print 'largest=',largest
    node.point=10
    calculate_heuristic_value(node,largest)

def calculate_heuristic_value(node,largest):
    global finalState
    #print '\n starting calculate_heuristic_value with largest= ',largest,' nodestate=',node.state
    if largest>0:

        for fpeg in finalState:
            if largest in fpeg:

                pos=finalState.index(fpeg)
     #           print largest, 'the largest is on peg no ', pos ,fpeg,' in finalstate. finalstate[pos]='#,finalstate[pos]
                if largest in node.state[pos]:
      #              print largest, 'the largest is on peg no ', pos ,' in node.state. node.state[pos]=',node.state[pos]
       #             print 'reducing point from ', node.point, ' to ', (node.point-1)
                    node.point=node.point-1
        #            print 'starting recursive with largest as ', largest-1
                    calculate_heuristic_value(node,largest-1)



def move(st1,st2):

    s1=st1[:]
    s2=st2[:]

    if len(s1)>0:
        topDisc=s1[len(s1)-1]
        lastofS2=len(s2)-1

        if len(s2)==0 or s2[lastofS2]>topDisc:
            s2.append(topDisc)
            s1.pop()

            return s1,s2
        else:
            return None
    else:
        return None


def moveDisc(n):
    global n_pegs
    stacks=[]

    for x in range(0,n_pegs):
        for y in range(0,n_pegs):

            #stacks=move(n.state[x],n.state[y])
            s1=n.state[x][:]
            s2=n.state[y][:]

            if len(s1)>0:
                topDisc=s1[len(s1)-1]
                lastofS2=len(s2)-1

                if len(s2)==0 or s2[lastofS2]>topDisc:
                    s2.append(topDisc)
                    s1.pop()

                    stacks = s1,s2
                else:
                    stacks = None
            else:
                stacks = None


            if stacks!=None:
                # print 'states after move', states
                nextnode=TOH()
                nextnode=deepcopy(n)
                nextnode.state[x]=deepcopy(stacks[0])
                nextnode.state[y]=deepcopy(stacks[1])


                # print 'states', states
                # print '\n'
                # print 'next node',nextnode.state
                if nextnode.state  in states:
                    #print 'nextnode in states'
                    a=1#dumb value
                else:
                    nodenumber=nextnode.nodeNumber
                    # print nextnode.state, 'next not in states'
                    states.append(nextnode.state)
                    return nextnode
    #print 'DEAD END'
    return None

def draw_peg(state):
    pattern = state
    return pattern

def toh_pattern(states):
    #[[], [], [], [5, 4, 3, 2, 1]]
    for state in states:
        drawPeg = ''
        state = str(state).replace('[','').replace(']','')
        #if state.isdigit() or state.find(',') != -1:
        if state.isdigit():
            drawPeg = draw_peg(state)
            drawPeg = disk(drawPeg)
        elif state.find(',') != -1:
            state = state.split(",")
            drawPeg = toh_pattern(state)
        print(drawPeg)
                

def back_tracing(node):
    print(color.BOLD+'\nBack Tracing'+color.END)
    while True:
        print('Selected Move ', node.nodeNumber,' :  ', node.state)
        #toh_pattern(node.state)
        if node.parent!=None:
            node=node.parent
        else:
            break


def BFS(node):
    global parentList,nodenumber,childList,target_reached,step

    print(color.PURPLE + color.BOLD + '\nSTEP : ' + str(step) + color.END)
    step+=1
    for node in parentList:
        if target_reached == False :
            print(color.RED + 'Final Move',node.nodeNumber,color.END+' :',node.state)
            exhausted=False
            parent=deepcopy(node)

            i=1
            while exhausted==False :

                i+=1
                childnode=moveDisc(node)

                if childnode!=None:
                    nodenumber+=1
                    childnode.nodeNumber=nodenumber
                    childnode.parent=node
                    parent.children.append(childnode)
                    childList.append(childnode)
                    print(color.YELLOW+'     Possible Move',childnode.nodeNumber,color.END+' :', childnode.state)
                    #print 'states', states
                    if childnode.state==finalState:
                        print(color.BOLD+color.GREEN+'\nFinal target reached'+color.END)
                        back_tracing(childnode)
                        target_reached=True


                else:
                    exhausted=True
    parentList=deepcopy(childList)
    childList=[]
    if target_reached==False :
        BFS(parentList)


def A_STAR():
    print('\n')
    global parentList,nodenumber,childList,target_reached,step,largestInTarget,largest
    leastPoint=10
    for node in parentList:
        #setPoints(node)
        node_evaluation(node)

        #print 'Node: ',node.nodeNumber, node.state,node.point
        if node.point<leastPoint:
            leastPoint=node.point


    for node in parentList:
        if target_reached == False and node.point==leastPoint:
            print(color.RED + '\nFinal Move :',node.nodeNumber,' :' + color.END,node.state, 
                  color.BOLD+color.GREEN+'\t\t h = ', node.point,color.END)
            exhausted=False
            parent=deepcopy(node)

            i=1
            while exhausted==False :

                i+=1
                childnode=moveDisc(node)

                if childnode!=None:
                    nodenumber+=1
                    childnode.nodeNumber=nodenumber
                    childnode.parent=node
                    parent.children.append(childnode)
                    childList.append(childnode)
                    print(color.YELLOW+'     Possible Move',childnode.nodeNumber,color.END+' :', childnode.state, 
                          color.BOLD+color.GREEN+'\t\t h = ', node.point,color.END)
                    #print 'states', states
                    if childnode.state==finalState:
                        print('Final target reached!')
                        back_tracing(childnode)
                        target_reached = True


                else:
                    exhausted=True
    parentList=deepcopy(childList)
    childList=[]
    if target_reached == False :
        A_STAR()



def prepare_pegs(n_disks, game_status):
    global n_pegs
    state=[]
    
    d = n_disks
    if(game_status == 1):
        for x in range(0 , n_pegs):
            if(x > 0):
                b = []
            else:
                b = [y for y in range(n_disks, 0, -1)]
            state.append(b)
    else:
        for x in range(0, n_pegs):
            if(x == n_pegs-1):
                f = [y for y in range(n_disks, 0, -1)]
            else:
                f = []
            state.append(f)
        
#     for x in range(0,n_pegs):
#         print('Disks in Peg',x+1,' : ',)
#         a = [int(x) for x in input().split()]
#         state.append(a)
    print(state)
    return state

n_pegs = 4

while True:
    print(color.PURPLE+color.BOLD+'\n\n# AI mini project by Alpesh Kumar and Vaibhav Kumar'+color.END)

    print('\n1. BFS Algorithm')
    print('2. A* Algorithm')
    print('3. Exit')


    choice = input("Please select the search algorithm --> ")


    if choice=='3':
        print('\nSorry to see you leaving!')
        quit()

    n_disks = 3
    n_disks = int(input("Please enter number of disks: "))
    
    print('\nEnter details for initial State')
    initialState = prepare_pegs(n_disks, game_status=1)
    print('\nEnter details for final State')
    finalState = prepare_pegs(n_disks, game_status=0)

    print('\nInitial state : ',initialState)
    print('Final states  : ',finalState)

    states=[]
    states=[initialState]
    nodenumber=1
    time=1
    target_reached = False

    node=TOH()
    node.state=initialState
    node.nodeNumber=nodenumber
    parentList=[node]
    childList=[]
    target_reached = False
    largestInTarget=False

    step=1


    if choice=='1':
        print(color.BOLD+'\nBFS Algorithm selected'+color.END)
        BFS(node)

    elif choice=='2':
        print(color.BOLD+'\nA* Algorithm selected'+color.END)
        A_STAR()

    elif choice=='3':
        print(color.BOLD+'\nSorry to see you leaving!'+color.END)
        quit()



# AI mini project by Alpesh Kumar and Vaibhav Kumar

1. BFS Algorithm
2. A* Algorithm
3. Exit
Please select the search algorithm --> 2
Please enter number of disks: 3

Enter details for initial State
[[3, 2, 1], [], [], []]

Enter details for final State
[[], [], [], [3, 2, 1]]

Initial state :  [[3, 2, 1], [], [], []]
Final states  :  [[], [], [], [3, 2, 1]]

A* Algorithm selected



Final Move : 1  : [[3, 2, 1], [], [], []] 		 h =  10 
     Possible Move 2  : [[3, 2], [1], [], []] 		 h =  10 
     Possible Move 3  : [[3, 2], [], [1], []] 		 h =  10 
     Possible Move 4  : [[3, 2], [], [], [1]] 		 h =  10 



Final Move : 2  : [[3, 2], [1], [], []] 		 h =  10 
     Possible Move 5  : [[3], [1], [2], []] 		 h =  10 
     Possible Move 6  : [[3], [1], [], [2]] 		 h =  10 

Final Move : 3  : [[3, 2], [], [1], []] 		 h =  10 
     Possible Move 7  : [[3], [2], [1], []] 		 h =  10 
     Possible Move 8  : [[3], [], [1], [2]] 		 h =  10 

Final Move : 4  : [[3, 2], [], [], [1]] 		 h =  1